In [1]:
# # Do not forget to check if your notebook log collection plugin is working
from mining_extension import check_logging
check_logging("http://3.249.245.244:9999")

Request successful!


In [2]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [3]:
train_features

,filename,cell_type,cell_number,execution_count,linesofcomment,linesofcode,variable_count,function_count,text/plain,image/png,...,helper_functions,load_data,data_exploration,data_preprocessing,evaluation,modelling,prediction,result_visualization,save_results,comment_only
0,nb_54880.ipynb,code,0,1,0.0,2.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,nb_54880.ipynb,code,1,2,1.0,4.0,4.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,nb_54880.ipynb,code,2,3,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,nb_54880.ipynb,code,3,4,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,nb_54880.ipynb,code,4,5,0.0,1.0,1.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,nb_95821.ipynb,code,27,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
5829,nb_95821.ipynb,code,29,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
5830,nb_95821.ipynb,code,31,-1,0.0,3.0,2.0,0.0,0,0,...,0,0,1,1,0,0,0,0,0,0
5831,nb_95821.ipynb,code,33,-1,5.0,17.0,14.0,0.0,0,0,...,0,0,0,0,1,1,1,0,0,0


In [4]:
train_features['primary_label']

0         helper_functions
1                load_data
2         data_exploration
3         data_exploration
4       data_preprocessing
               ...        
5828            evaluation
5829             modelling
5830    data_preprocessing
5831             modelling
5832            evaluation
Name: primary_label, Length: 5833, dtype: object

In [5]:
train_features.text[1]

["l_cols = ['user_id','movie_id','rating']",
 "r_cols = ['movie_id','title']",
 "l = pd.read_csv('u.data', sep='\\t', names=l_cols, usecols=range(3))",
 'r = pd.read_csv(\'u.item\', sep=\'|\', names=r_cols, usecols=range(2), encoding = "ISO-8859-1") ## contains unicode characters']

## Baseline

**I will build my model with CatBoost. So firstly let's create a model with this library without feature engineering and hyperparameters finding**

In [6]:
train_features['code_line_before']

0                                                       0
1                                    [import numpy as np]
2       [r = pd.read_csv('u.item', sep='|', names=r_co...
3                                              [l.head()]
4                                              [r.head()]
                              ...                        
5828                                         [plt.show()]
5829    [print("The average accuracy among all subject...
5830                                [srm.fit(movie_data)]
5831    [image_data_shared[subject] = stats.zscore(ima...
5832    [cm[subject] = cm[subject].astype('float') / c...
Name: code_line_before, Length: 5833, dtype: object

In [5]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score


def my_join(x):
    if isinstance(x, list):
        return ''.join(x)
    else:
        return 'SPECIALWORDFORSLAVA'

def my_len(it):
    if isinstance(it, list):
        return len(it)
    else:
        0

def create_new_text_feats(df, feature_name, max_len):
    for i in range(max_len):
        new_feature_name = f"{feature_name}_{i}"
        df[new_feature_name] = df[feature_name].apply(lambda x: x[i] if len(x) > i else 'SPECIALWORDFORSLAVA')
        
def add_small(s):
    if len(s) < const:
        return s + 'small'
    else:
        return s
const = 3

In [8]:
train_labels = train_features['primary_label']
train_features = train_features.drop(columns=['primary_label'])
# test_labels = test_features['primary_label']
# test_features = test_features.drop(columns=['primary_label'])
validation_labels = validation_features['primary_label']
validation_features = validation_features.drop(columns=['primary_label'])


train_features['code_line_before'] = train_features['code_line_before'].apply(my_join)
train_features['code_line_after'] = train_features['code_line_after'].apply(my_join)
train_features['markdown_heading'] = train_features['markdown_heading'].apply(my_join)
test_features['code_line_before'] = test_features['code_line_before'].apply(my_join)
test_features['code_line_after'] = test_features['code_line_after'].apply(my_join)
test_features['markdown_heading'] = test_features['markdown_heading'].apply(my_join)
validation_features['code_line_before'] = validation_features['code_line_before'].apply(my_join)
validation_features['code_line_after'] = validation_features['code_line_after'].apply(my_join)
validation_features['markdown_heading'] = validation_features['markdown_heading'].apply(my_join)


# max_len_text = max(train_features['text'].apply(my_len).max(),
#               validation_features['text'].apply(my_len).max(),
#               test_features['text'].apply(my_len).max())
# max_len_comment = max(train_features['comment'].apply(my_len).max(),
#               validation_features['comment'].apply(my_len).max(),
#               test_features['comment'].apply(my_len).max())
# max_len_packages = max(train_features['packages_info'].apply(my_len).max(),
#               validation_features['packages_info'].apply(my_len).max(),
#               test_features['packages_info'].apply(my_len).max())
max_len_text = 10
max_len_comment = 10
max_len_packages = 10


create_new_text_feats(train_features, 'comment', max_len_comment)
create_new_text_feats(train_features, 'text', max_len_text)
create_new_text_feats(train_features, 'packages_info', max_len_packages)

create_new_text_feats(test_features, 'comment', max_len_comment)
create_new_text_feats(test_features, 'text', max_len_text)
create_new_text_feats(test_features, 'packages_info', max_len_packages)

create_new_text_feats(validation_features, 'comment', max_len_comment)
create_new_text_feats(validation_features, 'text', max_len_text)
create_new_text_feats(validation_features, 'packages_info', max_len_packages)

In [9]:
categorical_feats = ['filename', 'cell_type', 'text/plain', 'image/png', 'text/html',
                     'execute_result', 'display_data', 'stream']
num_feats = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 'variable_count',
            'function_count', 'error']
primary_labels = ['helper_functions', 'load_data', 'data_preprocessing',
            'data_exploration', 'modelling', 'prediction', 'evaluation', 'result_visualization',
            'save_results', 'comment_only']
text_feats_fixed = ['text', 'comment', 'packages_info']
text_feats = [column for column in train_features.columns if column not in categorical_feats and column 
             not in num_feats and column not in text_feats_fixed and column not in primary_labels]

train_features = train_features.drop(columns=text_feats_fixed+primary_labels)
validation_features = validation_features.drop(columns=text_feats_fixed+primary_labels)
test_features = test_features.drop(columns=text_feats_fixed)

train_features[text_feats] = train_features[text_feats].applymap(add_small)
test_features[text_feats] = test_features[text_feats].applymap(add_small)
validation_features[text_feats] = validation_features[text_feats].applymap(add_small)

categorical_indices = [train_features.columns.get_loc(feature) for feature in categorical_feats]
text_indices = [train_features.columns.get_loc(feature) for feature in text_feats]

In [10]:
pool_train = Pool(train_features.values,
                  train_labels,
                  cat_features=categorical_indices,
                  text_features=text_indices)
pool_val = Pool(validation_features.values,
                  validation_labels,
                  cat_features=categorical_indices,
                  text_features=text_indices)
pool_test = Pool(test_features.values,
                  cat_features=categorical_indices,
                  text_features=text_indices)

In [12]:
catboost_clf = CatBoostClassifier(random_seed=3407, iterations=10,
                                 loss_function='MultiClass', custom_metric=['F1'])

catboost_clf.fit(
    pool_train,
    eval_set=pool_val
)

Learning rate set to 0.5
0:	learn: 1.6947700	test: 1.5744964	best: 1.5744964 (0)	total: 23.2s	remaining: 3m 29s
1:	learn: 1.4994446	test: 1.3845437	best: 1.3845437 (1)	total: 45.8s	remaining: 3m 3s
2:	learn: 1.3998656	test: 1.3660037	best: 1.3660037 (2)	total: 1m 5s	remaining: 2m 33s
3:	learn: 1.3386972	test: 1.3138035	best: 1.3138035 (3)	total: 1m 30s	remaining: 2m 15s
4:	learn: 1.3029897	test: 1.3014311	best: 1.3014311 (4)	total: 1m 54s	remaining: 1m 54s
5:	learn: 1.2473782	test: 1.2641125	best: 1.2641125 (5)	total: 2m 18s	remaining: 1m 32s
6:	learn: 1.2236949	test: 1.2607940	best: 1.2607940 (6)	total: 2m 47s	remaining: 1m 11s
7:	learn: 1.2148235	test: 1.2573158	best: 1.2573158 (7)	total: 3m 12s	remaining: 48.2s
8:	learn: 1.2003767	test: 1.2558896	best: 1.2558896 (8)	total: 3m 32s	remaining: 23.6s
9:	learn: 1.1952804	test: 1.2535650	best: 1.2535650 (9)	total: 3m 53s	remaining: 0us

bestTest = 1.253565015
bestIteration = 9



In [14]:
val_preds = catboost_clf.predict(pool_val)
f1 = f1_score(pool_val.get_label(), val_preds, average='weighted')
print("F1 score on the validation dataset:", f1)

F1 score on the validation dataset: 0.5134836817813787


## Let's place all text feats in one column with join

In [7]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [8]:
def my_join(x):
    if isinstance(x, list):
        return ''.join(x)
    else:
        return ''

In [9]:
train_labels = train_features['primary_label']
train_features = train_features.drop(columns=['primary_label'])
# test_labels = test_features['primary_label']
# test_features = test_features.drop(columns=['primary_label'])
validation_labels = validation_features['primary_label']
validation_features = validation_features.drop(columns=['primary_label'])


train_features['code_line_before'] = train_features['code_line_before'].apply(my_join)
train_features['code_line_after'] = train_features['code_line_after'].apply(my_join)
train_features['markdown_heading'] = train_features['markdown_heading'].apply(my_join)
train_features['comment'] = train_features['comment'].apply(my_join)
train_features['text'] = train_features['text'].apply(my_join)
train_features['packages_info'] = train_features['packages_info'].apply(my_join)

test_features['code_line_before'] = test_features['code_line_before'].apply(my_join)
test_features['code_line_after'] = test_features['code_line_after'].apply(my_join)
test_features['markdown_heading'] = test_features['markdown_heading'].apply(my_join)
test_features['comment'] = test_features['comment'].apply(my_join)
test_features['text'] = test_features['text'].apply(my_join)
test_features['packages_info'] = test_features['packages_info'].apply(my_join)

validation_features['code_line_before'] = validation_features['code_line_before'].apply(my_join)
validation_features['code_line_after'] = validation_features['code_line_after'].apply(my_join)
validation_features['markdown_heading'] = validation_features['markdown_heading'].apply(my_join)
validation_features['comment'] = validation_features['comment'].apply(my_join)
validation_features['text'] = validation_features['text'].apply(my_join)
validation_features['packages_info'] = validation_features['packages_info'].apply(my_join)

In [10]:
categorical_feats = ['filename', 'cell_type', 'text/plain', 'image/png', 'text/html',
                     'execute_result', 'display_data', 'stream']
num_feats = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 'variable_count',
            'function_count', 'error']
primary_labels = ['helper_functions', 'load_data', 'data_preprocessing',
            'data_exploration', 'modelling', 'prediction', 'evaluation', 'result_visualization',
            'save_results', 'comment_only']
# text_feats_fixed = ['text', 'comment', 'packages_info']
text_feats_fixed = []
text_feats = [column for column in train_features.columns if column not in categorical_feats and column 
             not in num_feats and column not in text_feats_fixed and column not in primary_labels]

train_features = train_features.drop(columns=text_feats_fixed+primary_labels)
validation_features = validation_features.drop(columns=text_feats_fixed+primary_labels)
test_features = test_features.drop(columns=text_feats_fixed)

# train_features[text_feats] = train_features[text_feats].applymap(add_small)
# test_features[text_feats] = test_features[text_feats].applymap(add_small)
# validation_features[text_feats] = validation_features[text_feats].applymap(add_small)

categorical_indices = [train_features.columns.get_loc(feature) for feature in categorical_feats]
text_indices = [train_features.columns.get_loc(feature) for feature in text_feats]

In [11]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score


pool_train = Pool(train_features.values,
                  train_labels,
                  cat_features=categorical_indices,
                  text_features=text_indices)
pool_val = Pool(validation_features.values,
                  validation_labels,
                  cat_features=categorical_indices,
                  text_features=text_indices)
pool_test = Pool(test_features.values,
                  cat_features=categorical_indices,
                  text_features=text_indices)

In [23]:
catboost_clf = CatBoostClassifier(random_seed=3407, iterations=50,
                                 loss_function='MultiClass', custom_metric=['F1'])

catboost_clf.fit(
    pool_train,
    eval_set=pool_val
)

Learning rate set to 0.351268
0:	learn: 1.7765573	test: 1.7208630	best: 1.7208630 (0)	total: 7.49s	remaining: 6m 7s
1:	learn: 1.5939216	test: 1.5138519	best: 1.5138519 (1)	total: 14.5s	remaining: 5m 47s
2:	learn: 1.4772468	test: 1.4022726	best: 1.4022726 (2)	total: 22.7s	remaining: 5m 54s
3:	learn: 1.4244295	test: 1.3390757	best: 1.3390757 (3)	total: 30.9s	remaining: 5m 55s
4:	learn: 1.4065807	test: 1.3194021	best: 1.3194021 (4)	total: 39.6s	remaining: 5m 56s
5:	learn: 1.3934001	test: 1.3104145	best: 1.3104145 (5)	total: 47s	remaining: 5m 44s
6:	learn: 1.3118043	test: 1.2966139	best: 1.2966139 (6)	total: 55.8s	remaining: 5m 42s
7:	learn: 1.2858073	test: 1.3013708	best: 1.2966139 (6)	total: 1m 3s	remaining: 5m 32s
8:	learn: 1.2524996	test: 1.2906858	best: 1.2906858 (8)	total: 1m 10s	remaining: 5m 20s
9:	learn: 1.2256546	test: 1.2708109	best: 1.2708109 (9)	total: 1m 17s	remaining: 5m 9s
10:	learn: 1.2103743	test: 1.2586300	best: 1.2586300 (10)	total: 1m 25s	remaining: 5m 1s
11:	learn: 1.

In [24]:
val_preds = catboost_clf.predict(pool_val)
f1 = f1_score(pool_val.get_label(), val_preds, average='weighted')
print("F1 score on the validation dataset:", f1)

F1 score on the validation dataset: 0.5806305370193506


In [13]:
catboost_clf = CatBoostClassifier(random_seed=3407, iterations=50,
                                 loss_function='MultiClass', custom_metric=['F1'])

catboost_clf.fit(
    pool_train,
    eval_set=pool_val
)

Learning rate set to 0.351268
0:	learn: 1.7408246	test: 1.6830217	best: 1.6830217 (0)	total: 7.24s	remaining: 5m 54s
1:	learn: 1.5951680	test: 1.5197003	best: 1.5197003 (1)	total: 13.9s	remaining: 5m 34s
2:	learn: 1.4867527	test: 1.4104175	best: 1.4104175 (2)	total: 21.8s	remaining: 5m 41s
3:	learn: 1.4392064	test: 1.3567033	best: 1.3567033 (3)	total: 29.6s	remaining: 5m 39s
4:	learn: 1.3506649	test: 1.3289917	best: 1.3289917 (4)	total: 37.6s	remaining: 5m 38s
5:	learn: 1.3132445	test: 1.3117655	best: 1.3117655 (5)	total: 45.8s	remaining: 5m 36s
6:	learn: 1.2809048	test: 1.2988045	best: 1.2988045 (6)	total: 52.9s	remaining: 5m 25s
7:	learn: 1.2746120	test: 1.2932885	best: 1.2932885 (7)	total: 59.2s	remaining: 5m 10s
8:	learn: 1.2641591	test: 1.2907018	best: 1.2907018 (8)	total: 1m 1s	remaining: 4m 40s
9:	learn: 1.2558509	test: 1.2904957	best: 1.2904957 (9)	total: 1m 12s	remaining: 4m 51s
10:	learn: 1.2293685	test: 1.2664406	best: 1.2664406 (10)	total: 1m 21s	remaining: 4m 49s
11:	learn

In [14]:
val_preds = catboost_clf.predict(pool_val)
f1 = f1_score(pool_val.get_label(), val_preds, average='weighted')
print("F1 score on the validation dataset:", f1)

F1 score on the validation dataset: 0.5558680737381004


**And optuna**

In [ ]:
import optuna


def objective(trial):
    iterations = trial.suggest_int("iterations", 10, 50)
    depth = trial.suggest_int("depth", 4, 12)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.5)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1, 10)

    # Train the CatBoost model with the suggested hyperparameters
    catboost_clf = CatBoostClassifier(
        random_seed=3407,
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        loss_function="MultiClass",
        custom_metric=["F1"],
        text_processing={
            "tokenizers": [
                {"tokenizer_id": "Space", "delimiter": " ", "separator_type": "ByDelimiter"},
            ],
            "dictionaries": [
                {
                    "dictionary_id": "Unigram",
                    "token_level_type": "Letter",
                    "occurrence_lower_bound": 50,
                },
            ],
            "feature_processing": {
                "default": [
                    {
                        "dictionaries_names": ["Unigram"],
                        "feature_calcers": ["TFIDF"],
                        "tokenizers_names": ["Space"],
                    },
                ],
            },
        },
    )
    catboost_clf.fit(pool_train, eval_set=pool_val)

    val_preds = catboost_clf.predict(pool_val)

    # Calculate the F1 score
    f1 = f1_score(pool_val.get_label(), val_preds, average="weighted")
    print('f1 is', f1)

    return f1

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout=3600)

print("Best hyperparameters:", study.best_params)

## What if nvm about text feats

In [3]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [4]:
train_labels = train_features['primary_label']
train_features = train_features.drop(columns=['primary_label'])
# test_labels = test_features['primary_label']
# test_features = test_features.drop(columns=['primary_label'])
validation_labels = validation_features['primary_label']
validation_features = validation_features.drop(columns=['primary_label'])

In [5]:
categorical_feats = ['filename', 'cell_type', 'text/plain', 'image/png', 'text/html',
                     'execute_result', 'display_data', 'stream']
num_feats = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 'variable_count',
            'function_count', 'error']
primary_labels = ['helper_functions', 'load_data', 'data_preprocessing',
            'data_exploration', 'modelling', 'prediction', 'evaluation', 'result_visualization',
            'save_results', 'comment_only']
# text_feats_fixed = ['text', 'comment', 'packages_info']
text_feats_fixed = []
text_feats = [column for column in train_features.columns if column not in categorical_feats and column 
             not in num_feats and column not in text_feats_fixed and column not in primary_labels]

train_features = train_features.drop(columns=text_feats+primary_labels)
validation_features = validation_features.drop(columns=text_feats+primary_labels)
test_features = test_features.drop(columns=text_feats)


categorical_indices = [train_features.columns.get_loc(feature) for feature in categorical_feats]

In [6]:
pool_train = Pool(train_features.values,
                  train_labels,
                  cat_features=categorical_indices)
pool_val = Pool(validation_features.values,
                  validation_labels,
                  cat_features=categorical_indices)
pool_test = Pool(test_features.values,
                  cat_features=categorical_indices)

NameError: name 'Pool' is not defined

In [31]:
catboost_clf = CatBoostClassifier(random_seed=3407, iterations=1000,
                                 loss_function='MultiClass', custom_metric=['F1'])

catboost_clf.fit(
    pool_train,
    eval_set=pool_val
)

Learning rate set to 0.111852
0:	learn: 2.0812827	test: 2.0623621	best: 2.0623621 (0)	total: 367ms	remaining: 6m 6s
1:	learn: 1.9531690	test: 1.9261761	best: 1.9261761 (1)	total: 585ms	remaining: 4m 52s
2:	learn: 1.8418119	test: 1.8046250	best: 1.8046250 (2)	total: 869ms	remaining: 4m 48s
3:	learn: 1.7647935	test: 1.7234716	best: 1.7234716 (3)	total: 1.27s	remaining: 5m 17s
4:	learn: 1.6956647	test: 1.6469255	best: 1.6469255 (4)	total: 1.57s	remaining: 5m 13s
5:	learn: 1.6370823	test: 1.5847633	best: 1.5847633 (5)	total: 1.82s	remaining: 5m 2s
6:	learn: 1.5932411	test: 1.5370458	best: 1.5370458 (6)	total: 2.12s	remaining: 5m
7:	learn: 1.5413152	test: 1.5114066	best: 1.5114066 (7)	total: 2.39s	remaining: 4m 56s
8:	learn: 1.4981060	test: 1.4892368	best: 1.4892368 (8)	total: 2.7s	remaining: 4m 57s
9:	learn: 1.4611839	test: 1.4518697	best: 1.4518697 (9)	total: 2.98s	remaining: 4m 54s
10:	learn: 1.4317896	test: 1.4450118	best: 1.4450118 (10)	total: 3.27s	remaining: 4m 53s
11:	learn: 1.40360

93:	learn: 0.9742421	test: 1.2161547	best: 1.2159208 (91)	total: 33.3s	remaining: 5m 21s
94:	learn: 0.9707215	test: 1.2137757	best: 1.2137757 (94)	total: 33.6s	remaining: 5m 20s
95:	learn: 0.9664835	test: 1.2114432	best: 1.2114432 (95)	total: 34s	remaining: 5m 19s
96:	learn: 0.9655407	test: 1.2109784	best: 1.2109784 (96)	total: 34.3s	remaining: 5m 19s
97:	learn: 0.9646026	test: 1.2111132	best: 1.2109784 (96)	total: 34.7s	remaining: 5m 19s
98:	learn: 0.9600163	test: 1.2106077	best: 1.2106077 (98)	total: 35.1s	remaining: 5m 19s
99:	learn: 0.9569697	test: 1.2097855	best: 1.2097855 (99)	total: 35.5s	remaining: 5m 19s
100:	learn: 0.9537572	test: 1.2091529	best: 1.2091529 (100)	total: 35.9s	remaining: 5m 19s
101:	learn: 0.9528522	test: 1.2084177	best: 1.2084177 (101)	total: 36.4s	remaining: 5m 20s
102:	learn: 0.9496527	test: 1.2107557	best: 1.2084177 (101)	total: 36.9s	remaining: 5m 21s
103:	learn: 0.9470561	test: 1.2109585	best: 1.2084177 (101)	total: 37.3s	remaining: 5m 20s
104:	learn: 0.9

184:	learn: 0.8284218	test: 1.2068504	best: 1.2053626 (131)	total: 1m 8s	remaining: 5m 3s
185:	learn: 0.8276442	test: 1.2069815	best: 1.2053626 (131)	total: 1m 9s	remaining: 5m 3s
186:	learn: 0.8269084	test: 1.2075011	best: 1.2053626 (131)	total: 1m 9s	remaining: 5m 2s
187:	learn: 0.8257524	test: 1.2078266	best: 1.2053626 (131)	total: 1m 10s	remaining: 5m 2s
188:	learn: 0.8250451	test: 1.2081741	best: 1.2053626 (131)	total: 1m 10s	remaining: 5m 2s
189:	learn: 0.8247848	test: 1.2082217	best: 1.2053626 (131)	total: 1m 10s	remaining: 5m 1s
190:	learn: 0.8245729	test: 1.2079016	best: 1.2053626 (131)	total: 1m 11s	remaining: 5m 1s
191:	learn: 0.8236589	test: 1.2088044	best: 1.2053626 (131)	total: 1m 11s	remaining: 5m
192:	learn: 0.8233319	test: 1.2090402	best: 1.2053626 (131)	total: 1m 11s	remaining: 5m
193:	learn: 0.8229263	test: 1.2089985	best: 1.2053626 (131)	total: 1m 12s	remaining: 5m
194:	learn: 0.8225615	test: 1.2088199	best: 1.2053626 (131)	total: 1m 12s	remaining: 5m
195:	learn: 0.

274:	learn: 0.7465136	test: 1.2145739	best: 1.2041613 (206)	total: 1m 41s	remaining: 4m 27s
275:	learn: 0.7459877	test: 1.2148451	best: 1.2041613 (206)	total: 1m 41s	remaining: 4m 27s
276:	learn: 0.7451017	test: 1.2146440	best: 1.2041613 (206)	total: 1m 42s	remaining: 4m 26s
277:	learn: 0.7449043	test: 1.2150744	best: 1.2041613 (206)	total: 1m 42s	remaining: 4m 26s
278:	learn: 0.7440868	test: 1.2163574	best: 1.2041613 (206)	total: 1m 42s	remaining: 4m 25s
279:	learn: 0.7429368	test: 1.2162139	best: 1.2041613 (206)	total: 1m 43s	remaining: 4m 25s
280:	learn: 0.7421181	test: 1.2151981	best: 1.2041613 (206)	total: 1m 43s	remaining: 4m 24s
281:	learn: 0.7417899	test: 1.2150189	best: 1.2041613 (206)	total: 1m 43s	remaining: 4m 24s
282:	learn: 0.7413227	test: 1.2163345	best: 1.2041613 (206)	total: 1m 44s	remaining: 4m 24s
283:	learn: 0.7407047	test: 1.2168948	best: 1.2041613 (206)	total: 1m 44s	remaining: 4m 23s
284:	learn: 0.7398278	test: 1.2170973	best: 1.2041613 (206)	total: 1m 44s	remain

364:	learn: 0.6832052	test: 1.2196021	best: 1.2041613 (206)	total: 2m 18s	remaining: 4m
365:	learn: 0.6829484	test: 1.2195913	best: 1.2041613 (206)	total: 2m 18s	remaining: 3m 59s
366:	learn: 0.6826843	test: 1.2197588	best: 1.2041613 (206)	total: 2m 18s	remaining: 3m 59s
367:	learn: 0.6816112	test: 1.2199670	best: 1.2041613 (206)	total: 2m 18s	remaining: 3m 58s
368:	learn: 0.6813011	test: 1.2201988	best: 1.2041613 (206)	total: 2m 19s	remaining: 3m 58s
369:	learn: 0.6803092	test: 1.2206646	best: 1.2041613 (206)	total: 2m 19s	remaining: 3m 57s
370:	learn: 0.6799897	test: 1.2203971	best: 1.2041613 (206)	total: 2m 20s	remaining: 3m 57s
371:	learn: 0.6797273	test: 1.2199972	best: 1.2041613 (206)	total: 2m 20s	remaining: 3m 57s
372:	learn: 0.6789438	test: 1.2196136	best: 1.2041613 (206)	total: 2m 21s	remaining: 3m 57s
373:	learn: 0.6780601	test: 1.2196151	best: 1.2041613 (206)	total: 2m 21s	remaining: 3m 56s
374:	learn: 0.6775193	test: 1.2197144	best: 1.2041613 (206)	total: 2m 21s	remaining:

...

635:	learn: 0.5205121	test: 1.2609959	best: 1.2041613 (206)	total: 4m 8s	remaining: 2m 22s
636:	learn: 0.5200483	test: 1.2614688	best: 1.2041613 (206)	total: 4m 9s	remaining: 2m 22s
637:	learn: 0.5196820	test: 1.2619099	best: 1.2041613 (206)	total: 4m 9s	remaining: 2m 21s
638:	learn: 0.5186966	test: 1.2623088	best: 1.2041613 (206)	total: 4m 10s	remaining: 2m 21s
639:	learn: 0.5185101	test: 1.2620675	best: 1.2041613 (206)	total: 4m 10s	remaining: 2m 21s
640:	learn: 0.5182089	test: 1.2625220	best: 1.2041613 (206)	total: 4m 11s	remaining: 2m 20s
641:	learn: 0.5176606	test: 1.2627609	best: 1.2041613 (206)	total: 4m 11s	remaining: 2m 20s
642:	learn: 0.5163058	test: 1.2643185	best: 1.2041613 (206)	total: 4m 12s	remaining: 2m 20s
643:	learn: 0.5158106	test: 1.2640551	best: 1.2041613 (206)	total: 4m 12s	remaining: 2m 19s
644:	learn: 0.5154491	test: 1.2640463	best: 1.2041613 (206)	total: 4m 13s	remaining: 2m 19s
645:	learn: 0.5148922	test: 1.2646937	best: 1.2041613 (206)	total: 4m 13s	remaining

725:	learn: 0.4756327	test: 1.2787092	best: 1.2041613 (206)	total: 4m 43s	remaining: 1m 47s
726:	learn: 0.4748670	test: 1.2787151	best: 1.2041613 (206)	total: 4m 44s	remaining: 1m 46s
727:	learn: 0.4747908	test: 1.2782918	best: 1.2041613 (206)	total: 4m 44s	remaining: 1m 46s
728:	learn: 0.4744745	test: 1.2783439	best: 1.2041613 (206)	total: 4m 45s	remaining: 1m 46s
729:	learn: 0.4743149	test: 1.2787983	best: 1.2041613 (206)	total: 4m 45s	remaining: 1m 45s
730:	learn: 0.4738444	test: 1.2777931	best: 1.2041613 (206)	total: 4m 45s	remaining: 1m 45s
731:	learn: 0.4731836	test: 1.2802088	best: 1.2041613 (206)	total: 4m 46s	remaining: 1m 44s
732:	learn: 0.4729451	test: 1.2804609	best: 1.2041613 (206)	total: 4m 46s	remaining: 1m 44s
733:	learn: 0.4724901	test: 1.2808105	best: 1.2041613 (206)	total: 4m 47s	remaining: 1m 44s
734:	learn: 0.4721093	test: 1.2813517	best: 1.2041613 (206)	total: 4m 47s	remaining: 1m 43s
735:	learn: 0.4715241	test: 1.2831497	best: 1.2041613 (206)	total: 4m 48s	remain

815:	learn: 0.4370277	test: 1.2974683	best: 1.2041613 (206)	total: 5m 17s	remaining: 1m 11s
816:	learn: 0.4365645	test: 1.2974464	best: 1.2041613 (206)	total: 5m 17s	remaining: 1m 11s
817:	learn: 0.4363907	test: 1.2970388	best: 1.2041613 (206)	total: 5m 18s	remaining: 1m 10s
818:	learn: 0.4361408	test: 1.2972982	best: 1.2041613 (206)	total: 5m 18s	remaining: 1m 10s
819:	learn: 0.4352231	test: 1.2976961	best: 1.2041613 (206)	total: 5m 18s	remaining: 1m 9s
820:	learn: 0.4349526	test: 1.2978843	best: 1.2041613 (206)	total: 5m 19s	remaining: 1m 9s
821:	learn: 0.4348437	test: 1.2976320	best: 1.2041613 (206)	total: 5m 19s	remaining: 1m 9s
822:	learn: 0.4339585	test: 1.2972832	best: 1.2041613 (206)	total: 5m 20s	remaining: 1m 8s
823:	learn: 0.4338288	test: 1.2976186	best: 1.2041613 (206)	total: 5m 20s	remaining: 1m 8s
824:	learn: 0.4337869	test: 1.2971596	best: 1.2041613 (206)	total: 5m 21s	remaining: 1m 8s
825:	learn: 0.4336284	test: 1.2984707	best: 1.2041613 (206)	total: 5m 21s	remaining: 1

906:	learn: 0.4008776	test: 1.3154021	best: 1.2041613 (206)	total: 5m 52s	remaining: 36.1s
907:	learn: 0.4007835	test: 1.3154939	best: 1.2041613 (206)	total: 5m 52s	remaining: 35.7s
908:	learn: 0.4004799	test: 1.3151700	best: 1.2041613 (206)	total: 5m 53s	remaining: 35.4s
909:	learn: 0.3997376	test: 1.3149347	best: 1.2041613 (206)	total: 5m 53s	remaining: 35s
910:	learn: 0.3995018	test: 1.3149365	best: 1.2041613 (206)	total: 5m 53s	remaining: 34.6s
911:	learn: 0.3989812	test: 1.3139593	best: 1.2041613 (206)	total: 5m 54s	remaining: 34.2s
912:	learn: 0.3984601	test: 1.3143112	best: 1.2041613 (206)	total: 5m 54s	remaining: 33.8s
913:	learn: 0.3983138	test: 1.3139515	best: 1.2041613 (206)	total: 5m 54s	remaining: 33.4s
914:	learn: 0.3978776	test: 1.3126359	best: 1.2041613 (206)	total: 5m 55s	remaining: 33s
915:	learn: 0.3975440	test: 1.3129972	best: 1.2041613 (206)	total: 5m 55s	remaining: 32.6s
916:	learn: 0.3974783	test: 1.3131164	best: 1.2041613 (206)	total: 5m 55s	remaining: 32.2s
917

997:	learn: 0.3683058	test: 1.3302401	best: 1.2041613 (206)	total: 6m 29s	remaining: 781ms
998:	learn: 0.3679371	test: 1.3301613	best: 1.2041613 (206)	total: 6m 30s	remaining: 390ms
999:	learn: 0.3676016	test: 1.3310752	best: 1.2041613 (206)	total: 6m 30s	remaining: 0us

bestTest = 1.204161271
bestIteration = 206

Shrink model to first 207 iterations.


In [32]:
val_preds = catboost_clf.predict(pool_val)
f1 = f1_score(pool_val.get_label(), val_preds, average='weighted')
print("F1 score on the validation dataset:", f1)

F1 score on the validation dataset: 0.5527753425974508


In [37]:
import optuna


def objective(trial):
    iterations = trial.suggest_int("iterations", 100, 1000)
    depth = trial.suggest_int("depth", 4, 12)
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.1)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1, 10)

    # Train the CatBoost model with the suggested hyperparameters
    catboost_clf = CatBoostClassifier(
        random_seed=3407,
        early_stopping_rounds=50,
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        loss_function="MultiClass",
        custom_metric=["F1"]
    )
    catboost_clf.fit(pool_train, eval_set=pool_val)

    val_preds = catboost_clf.predict(pool_val)

    # Calculate the F1 score
    f1 = f1_score(pool_val.get_label(), val_preds, average="weighted")
    print('f1 is', f1)

    return f1

In [38]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout=3600)

print("Best hyperparameters:", study.best_params)

[I 2023-05-06 19:56:21,764] A new study created in memory with name: no-name-f57f57f7-1733-4e10-8cf9-3f073068c153


0:	learn: 2.1259606	test: 2.1108427	best: 2.1108427 (0)	total: 346ms	remaining: 2m 40s
1:	learn: 2.0116704	test: 1.9900016	best: 1.9900016 (1)	total: 540ms	remaining: 2m 4s
2:	learn: 1.9086205	test: 1.8829216	best: 1.8829216 (2)	total: 1.28s	remaining: 3m 16s
3:	learn: 1.8197503	test: 1.7876350	best: 1.7876350 (3)	total: 1.93s	remaining: 3m 42s
4:	learn: 1.7538362	test: 1.7171495	best: 1.7171495 (4)	total: 2.73s	remaining: 4m 10s
5:	learn: 1.7032658	test: 1.6631096	best: 1.6631096 (5)	total: 3.37s	remaining: 4m 17s
6:	learn: 1.6551672	test: 1.6100576	best: 1.6100576 (6)	total: 3.66s	remaining: 3m 59s
7:	learn: 1.6061084	test: 1.5665510	best: 1.5665510 (7)	total: 4.33s	remaining: 4m 7s
8:	learn: 1.5640531	test: 1.5398644	best: 1.5398644 (8)	total: 4.67s	remaining: 3m 56s
9:	learn: 1.5255127	test: 1.5254990	best: 1.5254990 (9)	total: 5.24s	remaining: 3m 58s
10:	learn: 1.4934034	test: 1.4960186	best: 1.4960186 (10)	total: 5.79s	remaining: 3m 58s
11:	learn: 1.4627225	test: 1.4734389	best: 

93:	learn: 0.9860886	test: 1.2105276	best: 1.2105276 (93)	total: 1m 16s	remaining: 5m 2s
94:	learn: 0.9836155	test: 1.2115578	best: 1.2105276 (93)	total: 1m 17s	remaining: 5m
95:	learn: 0.9817899	test: 1.2107431	best: 1.2105276 (93)	total: 1m 18s	remaining: 4m 59s
96:	learn: 0.9806958	test: 1.2114576	best: 1.2105276 (93)	total: 1m 19s	remaining: 4m 59s
97:	learn: 0.9772206	test: 1.2110619	best: 1.2105276 (93)	total: 1m 19s	remaining: 4m 58s
98:	learn: 0.9752907	test: 1.2097912	best: 1.2097912 (98)	total: 1m 20s	remaining: 4m 57s
99:	learn: 0.9728587	test: 1.2084658	best: 1.2084658 (99)	total: 1m 21s	remaining: 4m 57s
100:	learn: 0.9681173	test: 1.2077500	best: 1.2077500 (100)	total: 1m 22s	remaining: 4m 57s
101:	learn: 0.9659177	test: 1.2080570	best: 1.2077500 (100)	total: 1m 23s	remaining: 4m 58s
102:	learn: 0.9628722	test: 1.2077203	best: 1.2077203 (102)	total: 1m 25s	remaining: 4m 59s
103:	learn: 0.9596908	test: 1.2056195	best: 1.2056195 (103)	total: 1m 25s	remaining: 4m 58s
104:	le

183:	learn: 0.7975803	test: 1.2035823	best: 1.2003966 (136)	total: 2m 36s	remaining: 3m 59s
184:	learn: 0.7970119	test: 1.2036741	best: 1.2003966 (136)	total: 2m 38s	remaining: 3m 59s
185:	learn: 0.7967436	test: 1.2036337	best: 1.2003966 (136)	total: 2m 39s	remaining: 3m 59s
186:	learn: 0.7963924	test: 1.2033852	best: 1.2003966 (136)	total: 2m 40s	remaining: 3m 59s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 1.200396565
bestIteration = 136

Shrink model to first 137 iterations.


[I 2023-05-06 19:59:03,396] Trial 0 finished with value: 0.552937947571961 and parameters: {'iterations': 465, 'depth': 8, 'learning_rate': 0.08979081640903386, 'l2_leaf_reg': 4.48485207118042}. Best is trial 0 with value: 0.552937947571961.


...

180:	learn: 0.7317626	test: 1.2147970	best: 1.2114554 (152)	total: 4m 3s	remaining: 7m 50s
181:	learn: 0.7293077	test: 1.2162406	best: 1.2114554 (152)	total: 4m 4s	remaining: 7m 48s
182:	learn: 0.7284232	test: 1.2173118	best: 1.2114554 (152)	total: 4m 5s	remaining: 7m 47s
183:	learn: 0.7275496	test: 1.2175431	best: 1.2114554 (152)	total: 4m 6s	remaining: 7m 45s
184:	learn: 0.7263930	test: 1.2193987	best: 1.2114554 (152)	total: 4m 8s	remaining: 7m 44s
185:	learn: 0.7244055	test: 1.2195650	best: 1.2114554 (152)	total: 4m 9s	remaining: 7m 42s
186:	learn: 0.7212965	test: 1.2191400	best: 1.2114554 (152)	total: 4m 10s	remaining: 7m 40s
187:	learn: 0.7191538	test: 1.2185962	best: 1.2114554 (152)	total: 4m 11s	remaining: 7m 38s
188:	learn: 0.7178967	test: 1.2187999	best: 1.2114554 (152)	total: 4m 12s	remaining: 7m 37s
189:	learn: 0.7158597	test: 1.2191555	best: 1.2114554 (152)	total: 4m 13s	remaining: 7m 35s
190:	learn: 0.7134363	test: 1.2198332	best: 1.2114554 (152)	total: 4m 14s	remaining: 7

[I 2023-05-06 20:04:39,347] Trial 2 finished with value: 0.547336284372258 and parameters: {'iterations': 531, 'depth': 9, 'learning_rate': 0.08772531424175346, 'l2_leaf_reg': 4.066042502354733}. Best is trial 1 with value: 0.5602852644944935.


f1 is 0.547336284372258
0:	learn: 2.1469296	test: 2.1340882	best: 2.1340882 (0)	total: 651ms	remaining: 2m 21s
1:	learn: 2.0407458	test: 2.0220351	best: 2.0220351 (1)	total: 1.22s	remaining: 2m 11s
2:	learn: 1.9473863	test: 1.9255654	best: 1.9255654 (2)	total: 9.75s	remaining: 11m 38s
3:	learn: 1.8555995	test: 1.8290526	best: 1.8290526 (3)	total: 17.6s	remaining: 15m 42s
4:	learn: 1.7883710	test: 1.7666148	best: 1.7666148 (4)	total: 25.4s	remaining: 18m 2s
5:	learn: 1.7291612	test: 1.7038344	best: 1.7038344 (5)	total: 33.3s	remaining: 19m 37s
6:	learn: 1.6710621	test: 1.6808243	best: 1.6808243 (6)	total: 39.5s	remaining: 19m 49s
7:	learn: 1.6226790	test: 1.6682593	best: 1.6682593 (7)	total: 47.7s	remaining: 20m 51s
8:	learn: 1.5769575	test: 1.6399825	best: 1.6399825 (8)	total: 56.3s	remaining: 21m 48s


[W 2023-05-06 20:05:38,814] Trial 3 failed with parameters: {'iterations': 218, 'depth': 12, 'learning_rate': 0.07599571959339498, 'l2_leaf_reg': 2.1815664165345745} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/Users/slavapirogov/jupyter-notebook-hackathon/venv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/mv/4srfnnv53bs0dtldb_0svy300000gn/T/ipykernel_10508/1347908422.py", line 21, in objective
    catboost_clf.fit(pool_train, eval_set=pool_val)
  File "/Users/slavapirogov/jupyter-notebook-hackathon/venv/lib/python3.9/site-packages/catboost/core.py", line 5128, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/Users/slavapirogov/jupyter-notebook-hackathon/venv/lib/python3.9/site-packages/catboost/core.py", line 2355, in _fit
    self._train(

KeyboardInterrupt: 

## TF-IDF

In [57]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [58]:
def my_join(x):
    if isinstance(x, list):
        return ''.join(x)
    else:
        return ''

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer


train_labels = train_features['primary_label']
train_features = train_features.drop(columns=['primary_label'])
# test_labels = test_features['primary_label']
# test_features = test_features.drop(columns=['primary_label'])
validation_labels = validation_features['primary_label']
validation_features = validation_features.drop(columns=['primary_label'])

In [60]:
from scipy.sparse import csr_matrix, hstack


categorical_feats = ['filename', 'cell_type', 'text/plain', 'image/png', 'text/html',
                     'execute_result', 'display_data', 'stream']
num_feats = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 'variable_count',
            'function_count', 'error']
primary_labels = ['helper_functions', 'load_data', 'data_preprocessing',
            'data_exploration', 'modelling', 'prediction', 'evaluation', 'result_visualization',
            'save_results', 'comment_only']
# text_feats_fixed = []
# text_feats = [column for column in train_features.columns if column not in categorical_feats and column 
#              not in num_feats and column not in text_feats_fixed and column not in primary_labels]

vectorizer = TfidfVectorizer()
tfidf_features = vectorizer.fit_transform(train_features['text'].apply(my_join))
train_features = train_features[num_feats]
non_text_sparse = csr_matrix(pd.get_dummies(train_features, sparse=True).values)
train_features = hstack([non_text_sparse, tfidf_features])

tfidf_features = vectorizer.transform(test_features['text'].apply(my_join))
test_features = test_features[num_feats]
non_text_sparse = csr_matrix(pd.get_dummies(test_features, sparse=True).values)
test_features = hstack([non_text_sparse, tfidf_features])

tfidf_features = vectorizer.transform(validation_features['text'].apply(my_join))
validation_features = validation_features[num_feats]
non_text_sparse = csr_matrix(pd.get_dummies(validation_features, sparse=True).values)
validation_features = hstack([non_text_sparse, tfidf_features])

In [61]:
pool_train = Pool(train_features,
                  train_labels)
pool_val = Pool(validation_features,
                  validation_labels)
pool_test = Pool(test_features)

In [64]:
catboost_clf = CatBoostClassifier(random_seed=3407, iterations=1000, early_stopping_rounds=100,
                                 loss_function='MultiClass', custom_metric=['F1'])

catboost_clf.fit(
    pool_train,
    eval_set=pool_val
)

Learning rate set to 0.111852
0:	learn: 2.0687773	test: 2.0608625	best: 2.0608625 (0)	total: 449ms	remaining: 7m 28s
1:	learn: 1.9041723	test: 1.8816538	best: 1.8816538 (1)	total: 1s	remaining: 8m 21s
2:	learn: 1.8015288	test: 1.7672360	best: 1.7672360 (2)	total: 1.55s	remaining: 8m 36s
3:	learn: 1.7156931	test: 1.6712178	best: 1.6712178 (3)	total: 2.18s	remaining: 9m 2s
4:	learn: 1.6455349	test: 1.5965394	best: 1.5965394 (4)	total: 2.64s	remaining: 8m 46s
5:	learn: 1.5959833	test: 1.5381751	best: 1.5381751 (5)	total: 3.21s	remaining: 8m 51s
6:	learn: 1.5532009	test: 1.4938475	best: 1.4938475 (6)	total: 3.65s	remaining: 8m 38s
7:	learn: 1.5193552	test: 1.4533298	best: 1.4533298 (7)	total: 4.09s	remaining: 8m 27s
8:	learn: 1.4691850	test: 1.4022207	best: 1.4022207 (8)	total: 4.51s	remaining: 8m 17s
9:	learn: 1.4409739	test: 1.3708618	best: 1.3708618 (9)	total: 4.95s	remaining: 8m 10s
10:	learn: 1.4112183	test: 1.3429129	best: 1.3429129 (10)	total: 5.39s	remaining: 8m 4s
11:	learn: 1.386

92:	learn: 0.9653124	test: 0.9997532	best: 0.9997532 (92)	total: 1m 7s	remaining: 10m 56s
93:	learn: 0.9636478	test: 0.9980280	best: 0.9980280 (93)	total: 1m 8s	remaining: 10m 55s
94:	learn: 0.9625542	test: 0.9974959	best: 0.9974959 (94)	total: 1m 8s	remaining: 10m 54s
95:	learn: 0.9605405	test: 0.9966834	best: 0.9966834 (95)	total: 1m 9s	remaining: 10m 54s
96:	learn: 0.9573519	test: 0.9940144	best: 0.9940144 (96)	total: 1m 10s	remaining: 10m 52s
97:	learn: 0.9560351	test: 0.9936165	best: 0.9936165 (97)	total: 1m 10s	remaining: 10m 50s
98:	learn: 0.9519127	test: 0.9903703	best: 0.9903703 (98)	total: 1m 11s	remaining: 10m 49s
99:	learn: 0.9496368	test: 0.9888303	best: 0.9888303 (99)	total: 1m 11s	remaining: 10m 47s
100:	learn: 0.9483796	test: 0.9882863	best: 0.9882863 (100)	total: 1m 12s	remaining: 10m 46s
101:	learn: 0.9445201	test: 0.9865602	best: 0.9865602 (101)	total: 1m 13s	remaining: 10m 45s
102:	learn: 0.9431436	test: 0.9863133	best: 0.9863133 (102)	total: 1m 14s	remaining: 10m 4

181:	learn: 0.8532071	test: 0.9450223	best: 0.9450223 (181)	total: 2m 26s	remaining: 10m 57s
182:	learn: 0.8525001	test: 0.9448158	best: 0.9448158 (182)	total: 2m 27s	remaining: 10m 57s
183:	learn: 0.8518262	test: 0.9445710	best: 0.9445710 (183)	total: 2m 28s	remaining: 10m 56s
184:	learn: 0.8511851	test: 0.9444953	best: 0.9444953 (184)	total: 2m 28s	remaining: 10m 55s
185:	learn: 0.8502573	test: 0.9438983	best: 0.9438983 (185)	total: 2m 29s	remaining: 10m 53s
186:	learn: 0.8492990	test: 0.9437158	best: 0.9437158 (186)	total: 2m 29s	remaining: 10m 52s
187:	learn: 0.8482745	test: 0.9432882	best: 0.9432882 (187)	total: 2m 30s	remaining: 10m 50s
188:	learn: 0.8477649	test: 0.9431728	best: 0.9431728 (188)	total: 2m 31s	remaining: 10m 49s
189:	learn: 0.8473716	test: 0.9431309	best: 0.9431309 (189)	total: 2m 32s	remaining: 10m 48s
190:	learn: 0.8465527	test: 0.9428299	best: 0.9428299 (190)	total: 2m 32s	remaining: 10m 47s
191:	learn: 0.8461556	test: 0.9427771	best: 0.9427771 (191)	total: 2m 

270:	learn: 0.8015715	test: 0.9294848	best: 0.9294848 (270)	total: 3m 38s	remaining: 9m 47s
271:	learn: 0.8011125	test: 0.9294534	best: 0.9294534 (271)	total: 3m 39s	remaining: 9m 46s
272:	learn: 0.8008915	test: 0.9294504	best: 0.9294504 (272)	total: 3m 39s	remaining: 9m 45s
273:	learn: 0.7991219	test: 0.9291950	best: 0.9291950 (273)	total: 3m 40s	remaining: 9m 44s
274:	learn: 0.7982713	test: 0.9286281	best: 0.9286281 (274)	total: 3m 41s	remaining: 9m 43s
275:	learn: 0.7977717	test: 0.9285584	best: 0.9285584 (275)	total: 3m 41s	remaining: 9m 42s
276:	learn: 0.7975007	test: 0.9285591	best: 0.9285584 (275)	total: 3m 42s	remaining: 9m 41s
277:	learn: 0.7966421	test: 0.9280193	best: 0.9280193 (277)	total: 3m 43s	remaining: 9m 39s
278:	learn: 0.7961294	test: 0.9280676	best: 0.9280193 (277)	total: 3m 44s	remaining: 9m 39s
279:	learn: 0.7957766	test: 0.9280774	best: 0.9280193 (277)	total: 3m 44s	remaining: 9m 38s
280:	learn: 0.7954584	test: 0.9278939	best: 0.9278939 (280)	total: 3m 45s	remain

360:	learn: 0.7615176	test: 0.9188333	best: 0.9188333 (360)	total: 4m 52s	remaining: 8m 37s
361:	learn: 0.7612630	test: 0.9188410	best: 0.9188333 (360)	total: 4m 53s	remaining: 8m 37s
362:	learn: 0.7607565	test: 0.9186498	best: 0.9186498 (362)	total: 4m 54s	remaining: 8m 36s
363:	learn: 0.7605554	test: 0.9186250	best: 0.9186250 (363)	total: 4m 55s	remaining: 8m 36s
364:	learn: 0.7593426	test: 0.9180058	best: 0.9180058 (364)	total: 4m 55s	remaining: 8m 34s
365:	learn: 0.7590280	test: 0.9179933	best: 0.9179933 (365)	total: 4m 56s	remaining: 8m 33s
366:	learn: 0.7588189	test: 0.9180036	best: 0.9179933 (365)	total: 4m 57s	remaining: 8m 32s
367:	learn: 0.7586900	test: 0.9180179	best: 0.9179933 (365)	total: 4m 57s	remaining: 8m 31s
368:	learn: 0.7582919	test: 0.9179964	best: 0.9179933 (365)	total: 4m 58s	remaining: 8m 30s
369:	learn: 0.7580487	test: 0.9181438	best: 0.9179933 (365)	total: 4m 58s	remaining: 8m 29s
370:	learn: 0.7577198	test: 0.9181226	best: 0.9179933 (365)	total: 4m 59s	remain

...

630:	learn: 0.6911970	test: 0.9011213	best: 0.9010027 (620)	total: 8m 53s	remaining: 5m 12s
631:	learn: 0.6907389	test: 0.9009111	best: 0.9009111 (631)	total: 8m 54s	remaining: 5m 11s
632:	learn: 0.6902473	test: 0.9006494	best: 0.9006494 (632)	total: 8m 55s	remaining: 5m 10s
633:	learn: 0.6900546	test: 0.9006476	best: 0.9006476 (633)	total: 8m 55s	remaining: 5m 9s
634:	learn: 0.6898429	test: 0.9002764	best: 0.9002764 (634)	total: 8m 56s	remaining: 5m 8s
635:	learn: 0.6895643	test: 0.8998935	best: 0.8998935 (635)	total: 8m 57s	remaining: 5m 7s
636:	learn: 0.6894211	test: 0.8999064	best: 0.8998935 (635)	total: 8m 58s	remaining: 5m 6s
637:	learn: 0.6893068	test: 0.8998928	best: 0.8998928 (637)	total: 8m 59s	remaining: 5m 5s
638:	learn: 0.6891866	test: 0.8999413	best: 0.8998928 (637)	total: 9m	remaining: 5m 5s
639:	learn: 0.6890681	test: 0.8999008	best: 0.8998928 (637)	total: 9m 1s	remaining: 5m 4s
640:	learn: 0.6888685	test: 0.8997484	best: 0.8997484 (640)	total: 9m 2s	remaining: 5m 3s
64

720:	learn: 0.6709208	test: 0.8966324	best: 0.8965931 (709)	total: 10m 16s	remaining: 3m 58s
721:	learn: 0.6707859	test: 0.8964317	best: 0.8964317 (721)	total: 10m 17s	remaining: 3m 57s
722:	learn: 0.6706965	test: 0.8964761	best: 0.8964317 (721)	total: 10m 18s	remaining: 3m 56s
723:	learn: 0.6703873	test: 0.8963001	best: 0.8963001 (723)	total: 10m 19s	remaining: 3m 56s
724:	learn: 0.6702981	test: 0.8962691	best: 0.8962691 (724)	total: 10m 20s	remaining: 3m 55s
725:	learn: 0.6701602	test: 0.8962618	best: 0.8962618 (725)	total: 10m 21s	remaining: 3m 54s
726:	learn: 0.6699166	test: 0.8961010	best: 0.8961010 (726)	total: 10m 22s	remaining: 3m 53s
727:	learn: 0.6697845	test: 0.8960866	best: 0.8960866 (727)	total: 10m 23s	remaining: 3m 53s
728:	learn: 0.6694939	test: 0.8962971	best: 0.8960866 (727)	total: 10m 25s	remaining: 3m 52s
729:	learn: 0.6692908	test: 0.8962864	best: 0.8960866 (727)	total: 10m 28s	remaining: 3m 52s
730:	learn: 0.6691669	test: 0.8961850	best: 0.8960866 (727)	total: 10m

809:	learn: 0.6576349	test: 0.8949725	best: 0.8948756 (802)	total: 11m 53s	remaining: 2m 47s
810:	learn: 0.6574693	test: 0.8949426	best: 0.8948756 (802)	total: 11m 54s	remaining: 2m 46s
811:	learn: 0.6572783	test: 0.8949678	best: 0.8948756 (802)	total: 11m 55s	remaining: 2m 45s
812:	learn: 0.6571486	test: 0.8949649	best: 0.8948756 (802)	total: 11m 56s	remaining: 2m 44s
813:	learn: 0.6567157	test: 0.8952645	best: 0.8948756 (802)	total: 11m 57s	remaining: 2m 43s
814:	learn: 0.6565498	test: 0.8952451	best: 0.8948756 (802)	total: 11m 58s	remaining: 2m 42s
815:	learn: 0.6561134	test: 0.8951574	best: 0.8948756 (802)	total: 11m 59s	remaining: 2m 42s
816:	learn: 0.6559986	test: 0.8951441	best: 0.8948756 (802)	total: 12m 2s	remaining: 2m 41s
817:	learn: 0.6559134	test: 0.8951213	best: 0.8948756 (802)	total: 12m 3s	remaining: 2m 40s
818:	learn: 0.6558004	test: 0.8951028	best: 0.8948756 (802)	total: 12m 4s	remaining: 2m 40s
819:	learn: 0.6556938	test: 0.8950969	best: 0.8948756 (802)	total: 12m 4s

898:	learn: 0.6437455	test: 0.8922113	best: 0.8921730 (897)	total: 13m 5s	remaining: 1m 28s
899:	learn: 0.6435848	test: 0.8922973	best: 0.8921730 (897)	total: 13m 5s	remaining: 1m 27s
900:	learn: 0.6433164	test: 0.8918095	best: 0.8918095 (900)	total: 13m 6s	remaining: 1m 26s
901:	learn: 0.6432282	test: 0.8917934	best: 0.8917934 (901)	total: 13m 6s	remaining: 1m 25s
902:	learn: 0.6429408	test: 0.8919250	best: 0.8917934 (901)	total: 13m 7s	remaining: 1m 24s
903:	learn: 0.6428777	test: 0.8919242	best: 0.8917934 (901)	total: 13m 8s	remaining: 1m 23s
904:	learn: 0.6425714	test: 0.8921479	best: 0.8917934 (901)	total: 13m 8s	remaining: 1m 22s
905:	learn: 0.6424500	test: 0.8921108	best: 0.8917934 (901)	total: 13m 9s	remaining: 1m 21s
906:	learn: 0.6423167	test: 0.8920607	best: 0.8917934 (901)	total: 13m 9s	remaining: 1m 20s
907:	learn: 0.6422569	test: 0.8920480	best: 0.8917934 (901)	total: 13m 10s	remaining: 1m 20s
908:	learn: 0.6421271	test: 0.8920743	best: 0.8917934 (901)	total: 13m 11s	rema

988:	learn: 0.6302986	test: 0.8907726	best: 0.8907726 (988)	total: 14m 12s	remaining: 9.48s
989:	learn: 0.6301841	test: 0.8907492	best: 0.8907492 (989)	total: 14m 12s	remaining: 8.61s
990:	learn: 0.6301017	test: 0.8907576	best: 0.8907492 (989)	total: 14m 13s	remaining: 7.75s
991:	learn: 0.6299957	test: 0.8907627	best: 0.8907492 (989)	total: 14m 13s	remaining: 6.89s
992:	learn: 0.6299363	test: 0.8907692	best: 0.8907492 (989)	total: 14m 14s	remaining: 6.02s
993:	learn: 0.6296574	test: 0.8905441	best: 0.8905441 (993)	total: 14m 15s	remaining: 5.16s
994:	learn: 0.6293113	test: 0.8896853	best: 0.8896853 (994)	total: 14m 15s	remaining: 4.3s
995:	learn: 0.6292225	test: 0.8896925	best: 0.8896853 (994)	total: 14m 16s	remaining: 3.44s
996:	learn: 0.6290986	test: 0.8897352	best: 0.8896853 (994)	total: 14m 16s	remaining: 2.58s
997:	learn: 0.6289867	test: 0.8897360	best: 0.8896853 (994)	total: 14m 17s	remaining: 1.72s
998:	learn: 0.6289249	test: 0.8897560	best: 0.8896853 (994)	total: 14m 17s	remain

In [66]:
val_preds = catboost_clf.predict(pool_val)
f1 = f1_score(pool_val.get_label(), val_preds, average='weighted')
print("F1 score on the validation dataset:", f1)

F1 score on the validation dataset: 0.699410887261825


In [67]:
test_preds = catboost_clf.predict(pool_test)

preds_df = pd.DataFrame(test_preds, columns=["prediction"])

preds_df.to_csv("submission.csv", index=False)

In [69]:
preds_df

,prediction
0,helper_functions
1,data_preprocessing
2,data_preprocessing
3,data_preprocessing
4,data_exploration
...,...
1913,data_exploration
1914,prediction
1915,prediction
1916,data_preprocessing


In [71]:
!git status

Текущая ветка: main
Эта ветка соответствует «origin/main».

Изменения, которые не в индексе для коммита:
  (используйте «git add <файл>...», чтобы добавить файл в индекс)
  (используйте «git restore <файл>...», чтобы отменить изменения в рабочем каталоге)
	изменено:      task1.ipynb
	изменено:      task2.ipynb

Неотслеживаемые файлы:
  (используйте «git add <файл>...», чтобы добавить в то, что будет включено в коммит)
	.idea/
	.ipynb_checkpoints/
	36c724d9-3a5f-43e2-aba2-2d0933046f35.log
	8a599028-9e81-4862-ad7f-15500e742f27.log
	all_data.csv
	catboost_info/
	jetnet_logs.txt
	mean_actions_by_user_in_day.csv
	submission.csv

индекс пуст (используйте «git add» и/или «git commit -a»)
